# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import pandas as pd 

##  Builting a dataframe of the postal code of each neighborhood

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_list=pd.read_html(url,header=0)
df=df_list[0]
df.head()
df.shape

(180, 3)

### Deleting any row containing Not assigned in "Borough"

In [3]:
df_1=df.drop(df[df['Borough']=='Not assigned'].index).reset_index(drop=True)
df_1.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Making Not assigned "Neighborhood" to be the same as the respective "Borough"

In [4]:
rows_NA=df_1.loc[df_1.Neighborhood == 'Not assigned'].index
df_1['Neighborhood'][rows_NA]=df_1['Borough'][rows_NA]
df_1.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Inserting multiple "Neighborhood"s in a single row separated by commas having same "Postal Code"

In [5]:
df_2=df_1.groupby("Postal Code").agg(lambda x:','.join(set(x))).reset_index()


In [6]:
df_2.shape

(103, 3)